In [3]:
import findspark
findspark.init()
findspark.find()

'/Users/benplatten/Applications/spark-3.1.1-bin-hadoop2.7'

In [21]:
from pyspark import SparkConf, SparkContext
import collections
import re

In [5]:
conf = SparkConf().setMaster("local").setAppName("AverageRating")
sc = SparkContext(conf = conf)

In [6]:
data_folder = '/Users/benplatten/workspace/UvA_BD/bigdata/data/'

### Movies / Genres
##### id title genres

In [63]:
lines = sc.textFile(data_folder+"/ml-latest-small/movies.csv")

In [78]:
def loadMovieGenres():
    movieGenres = {}
    with open(data_folder + "ml-latest-small/movies.csv", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split(';')
            genres = fields[2]
            genres = genres.strip('\n')
            genres = genres.split("|")
            movieGenres[int(fields[0])] = genres
    return movieGenres

In [80]:
id_genre_bc = sc.broadcast(loadMovieGenres())

### Ratings
##### userId movieId rating timestamp

In [120]:
rating_lines = sc.textFile(data_folder+"/ml-latest-small/ratings.csv")

In [121]:
def parseLine(line):
    fields =line.split(';')
    movieId = int(fields[1])
    rating= float(fields[2])
    return (movieId, rating)

In [122]:
ratings_rdd = rating_lines.map(parseLine)

In [123]:
def broadcastJoin(movieId,rating):
    return (movieId,id_genre_bc.value[movieId],rating)

In [124]:
id_genres_rating = rdd2.map(lambda x: broadcastJoin(x[0],x[1]))

In [132]:
genre_ratings = id_genres_rating.flatMap(lambda x: [(w, x[2]) for w in x[1]]).collect()
g_rdd = sc.parallelize(genre_ratings)
ratingsByGenre = g_rdd.mapValues(lambda x: (x,1)).reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1]))
averagesByGenre = ratingsByGenre.mapValues(lambda x: x[0] / x[1])
results = averagesByGenre.collect()

In [134]:
for result in results:
    print(result)

('Adventure', 3.5086089151939075)
('Animation', 3.6300271933590955)
('Children', 3.412956125108601)
('Comedy', 3.3847207640898267)
('Fantasy', 3.4910005070136894)
('Romance', 3.5065107040388437)
('Action', 3.447984331646809)
('Crime', 3.658293867274144)
('Thriller', 3.4937055799183425)
('Mystery', 3.632460255407871)
('Horror', 3.258195034974626)
('Drama', 3.6562000620125454)
('War', 3.8082938876312)
('Western', 3.583937823834197)
('Sci-Fi', 3.455747593086649)
('Musical', 3.5636781053649105)
('Film-Noir', 3.920114942528736)
('IMAX', 3.618335343787696)
('Documentary', 3.797785069729286)
('(no genres listed)', 3.4893617021276597)
('', 3.0)
